<a href="https://colab.research.google.com/github/tolgagonen/multilabeled-turkish-textclassification-tensorflow/blob/main/multilabeled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

In [127]:
import sys
!pip install -q ftfy
sys.setrecursionlimit(10000)
import ftfy


In [134]:
df = pd.read_csv("7allV03.csv",encoding='UTF-8')
#latin-1 kullandığımda saçma sapan bozulmalar olduğu için yukarıda ki kod parçası ile türkçe karakterleri katıp utf-8 kullandım

In [129]:
df.shape

(4900, 2)

In [136]:
df.head()

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,siyaset,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,siyaset,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek ...


In [137]:
print((df.category == "siyaset ").sum()) # Siyaset ile ilgili cümleler
print((df.category == "ekonomi ").sum()) # Ekonomi ile ilgili cümleler
print((df.category == "kultur ").sum()) # kultur ile ilgili cümleler
print((df.category == "dunya ").sum()) # dunya ile ilgili cümleler
print((df.category == "spor ").sum()) # spor ile ilgili cümleler
print((df.category == "saglik ").sum()) # saglik ile ilgili cümleler
print((df.category == "teknoloji ").sum()) # teknoloji ile ilgili cümleler

700
700
700
700
700
700
700


In [138]:
# Tekrar işleme , burada remove_ULR de linkleri yok ediyoruz
#altındaki punct kısmında ise noktaları
import re
import string

def remove_URL(text):
    url =re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", str(text))

def remove_punct(text):
    translator = str.maketrans("","", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [139]:
df["text"] = df.text.map(remove_URL)
df["text"] = df.text.map(remove_punct)

In [140]:
#Twitter gibi uygulamalarda genellikle noktalama işaretlerine çok dikkat edilmez
#Bu yüzden cümlelerin sonunda stopwords dediğimiz cümleleri bitirme kelimeleri bulunur
#Bunlardan burada kurtulmak için hazırlanıyoruz.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#stop words A stop words is a commonly used word (such as "the" , "a")

stop= set(stopwords.words("english"))
stop2= set(stopwords.words("turkish"))

def remove_stopwords(text):
  filtered_words = [word.lower() for word in text.split() if word.lower() not in stop2]
  return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [141]:
df["text"] = df.text.map(remove_stopwords)

In [142]:
df.text

0       3 milyon ön seçim vaadi mhp nin 10 olağan büyü...
1       mesutyılmaz yücedivan ceza alabilirdi prof dr ...
2       disko lar kaldırılıyor başbakanyardımcısı arın...
3       sarıgül anayasamahkemesi gidiyor mustafasarıgü...
4       erdoğan idamın bir haklılık sebebi var demek y...
                              ...                        
4895    iphone lara geri dönüyor ios 6 sürümüyle telef...
4896    muslukta devrim sadece elimizi yıkadığımız mus...
4897    halka iyi anlatılmalı bilgiteknolojileri ileti...
4898    çöpe gidiyorlar apple 775 bin uygulamayla apps...
4899    google edwardgorey dedi google ünlü illüstratö...
Name: text, Length: 4900, dtype: object

In [143]:
#Burada tüm farklı kelimeleri sayıyoruz
from collections import Counter

def counter_word(text_col):
  count = Counter()
  for text in text_col.values:
    for word in text.split():
      count[word] += 1
  return count

counter = counter_word(df.text)

In [144]:
len(counter)

110898

In [145]:
counter.most_common(5) 

[('bir', 20463),
 ('\x92', 11924),
 ('türkiye', 5475),
 ('nin', 5071),
 ('olarak', 5058)]

In [146]:
num_unique_words = len(counter) # Eşsiz Kelime Sayısı

In [147]:
#burada elimdeki category datasındaki 7 farklı kategori için 1-7 değerler veriyorum

df.loc[df['category'] == "siyaset", 'category'] = 1
df.loc[df['category'] == "ekonomi", 'category'] = 2
df.loc[df['category'] == "kultur", 'category'] = 3
df.loc[df['category'] == "dunya", 'category'] = 4
df.loc[df['category'] == "spor", 'category'] = 5
df.loc[df['category'] == "saglik", 'category'] = 6
df.loc[df['category'] == "teknoloji", 'category'] = 7
df.loc[df['category'] == "teknol", 'category'] = 7


In [148]:
df.head()

,category,text
0,siyaset,3 milyon ön seçim vaadi mhp nin 10 olağan büyü...
1,siyaset,mesutyılmaz yücedivan ceza alabilirdi prof dr ...
2,siyaset,disko lar kaldırılıyor başbakanyardımcısı arın...
3,siyaset,sarıgül anayasamahkemesi gidiyor mustafasarıgü...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek y...


In [195]:
from sklearn.utils import shuffle

# Verileri shuffle yapma
shuffled_df = shuffle(df)

In [197]:
shuffled_df.head(

)

,category,text
2681,kultur,bifo yu alipoyrazoğlu yönetecek ilki 2006 yılı...
4134,spor,ünlü forvet polis dövdü ingiltere premierlig e...
823,dunya,olağanüstü hal 3 ay uzatıldı tunus cumhurbaşka...
1296,dunya,japonya erken genel seçime gidiyor japonya hük...
2779,kultur,srebrenica inferno dünya prömiyeri ankara yapı...


In [196]:
#Eğitim için elimizdeki datanın yüzde 65'ini kullanıcam
#Geri kalan ise valdiation için kullanıcam
train_size = int(shuffled_df.shape[0] * 0.65)
train_df = shuffled_df[:train_size]
val_df = shuffled_df[train_size:]

#Textleri ve type 'ları ayırıyorum
train_sentences = train_df['text'].to_numpy()
train_labels = train_df['category'].to_numpy()
val_sentences = val_df['text'].to_numpy()
val_labels = val_df['category'].to_numpy()

In [198]:
train_sentences.shape , val_sentences.shape , train_labels.shape

((3185,), (1715,), (3185,))

In [199]:
train_labels

array(['kultur ', 'spor ', 'dunya ', ..., 'dunya ', 'kultur ', 'spor '],
      dtype=object)

In [200]:
#tokenizer paragrafları ve cümleleri daha kolay anlam atanabilecek daha küçük birimlere ayırır

from tensorflow.keras.preprocessing.text import Tokenizer

#vectorize a text corpus by turning each text into a sequance of integers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences)

In [201]:
#Her kelimenin kendine özel bir indexi var
word_index = tokenizer.word_index

In [202]:
word_index

{'bir': 1,
 '\x92': 2,
 'türkiye': 3,
 'nin': 4,
 'olarak': 5,
 '\x94': 6,
 '\x93': 7,
 'olan': 8,
 'ın': 9,
 'in': 10,
 'olduğunu': 11,
 'dedi': 12,
 '1': 13,
 'nın': 14,
 'yüzde': 15,
 '2': 16,
 '3': 17,
 'kadar': 18,
 'ilk': 19,
 'büyük': 20,
 'yeni': 21,
 'sonra': 22,
 'bin': 23,
 'a': 24,
 '5': 25,
 'son': 26,
 'söyledi': 27,
 'yıl': 28,
 'göre': 29,
 'önemli': 30,
 'var': 31,
 'ilgili': 32,
 'milyon': 33,
 '4': 34,
 'etti': 35,
 'olduğu': 36,
 'e': 37,
 'tarafından': 38,
 'iyi': 39,
 'yer': 40,
 'ifade': 41,
 'devam': 42,
 'iki': 43,
 'eden': 44,
 'ancak': 45,
 'yapılan': 46,
 '10': 47,
 'önce': 48,
 'ye': 49,
 'arasında': 50,
 'konuştu': 51,
 'istanbul': 52,
 'un': 53,
 'oldu': 54,
 '6': 55,
 'türk': 56,
 'dr': 57,
 'değil': 58,
 'ardından': 59,
 'fazla': 60,
 'yaptığı': 61,
 'nde': 62,
 'dan': 63,
 'sadece': 64,
 'içinde': 65,
 'den': 66,
 'aynı': 67,
 '2012': 68,
 'nun': 69,
 'ortaya': 70,
 'abd': 71,
 'özel': 72,
 'zaman': 73,
 'şöyle': 74,
 'geçen': 75,
 'şekilde': 76,
 'bir

In [203]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [204]:
print(train_sentences[10:15])
print(train_sequences[10:15])

['güvenç dağüstün urban lounge 18 11 2012 22 00 ankara \x92 dan viyana \x92 uzanan akademik kariyerini yurtiçinde yurtdışında yer aldığı başarılı projelerle sürdürdü avusturya \x92 c orff \x92 un \x93 carmina burana \x94 adlı eserinde bariton sololarının yanı sıra kontrtenor solosunu seslendirerek bir ilke imza attı viyana \x92 seslendirdiği yunan besteci perikles liakakis \x92 in \x93 çizmeli kedi \x94 operasındaki başrol eserin dünyada ilk seslendirilişinin solistlerinden oldu sertaberener ortak projeleri olan \x93 turco \x96 latino connection \x94 caz konserleri büyük beğeni topladı fazılsay \x92 ın \x93 nazım \x94 eserini fazılsay gencoerkal sertaberener birlikte seslendirdi \x93 rumeli hisarı \x96 yıldızlı geceler \x94 konserlerine fazılsay birlikte bir resital verdi say \x92 ın şarkılarının türkiye \x92 ilk seslendirilişini gerçekleştirdi son olarak şef ibrahimyazıcı yönetimindeki bilkent senfoni orkestrası eşliğinde say \x92 ın \x93 nazım \x94 adlı eserinin cd dvd kayıtlarını fa

In [205]:
# dizileri aynı uzunluğa sahip olacak şekilde doldurun
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length=20
 
train_padded= pad_sequences(train_sequences , maxlen=max_length , padding="post",truncating="post")
val_padded= pad_sequences(val_sequences, maxlen=max_length, padding="post" , truncating="post")
train_padded.shape , val_padded.shape

((3185, 20), (1715, 20))

In [206]:
train_padded[10]

array([ 9451, 26694, 50463, 21712,   239,   165,    68,   434,   212,
         207,     2,    63,  6319,     2,  3128,  4565, 15842, 26695,
        3564,    40], dtype=int32)

In [207]:
val_padded[10]

array([62596,     2,    69,  1838, 37840,   270,   116,     2,  1493,
        7380,    30, 12225, 41812,     2,    69, 14092,     2,  8301,
          61,     1], dtype=int32)

In [208]:
train_labels[1481]

'kultur '

In [209]:
val_labels[1481]

'dunya '

In [210]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

güvenç dağüstün urban lounge 18 11 2012 22 00 ankara  dan viyana  uzanan akademik kariyerini yurtiçinde yurtdışında yer aldığı başarılı projelerle sürdürdü avusturya  c orff  un  carmina burana  adlı eserinde bariton sololarının yanı sıra kontrtenor solosunu seslendirerek bir ilke imza attı viyana  seslendirdiği yunan besteci perikles liakakis  in  çizmeli kedi  operasındaki başrol eserin dünyada ilk seslendirilişinin solistlerinden oldu sertaberener ortak projeleri olan  turco  latino connection  caz konserleri büyük beğeni topladı fazılsay  ın  nazım  eserini fazılsay gencoerkal sertaberener birlikte seslendirdi  rumeli hisarı  yıldızlı geceler  konserlerine fazılsay birlikte bir resital verdi say  ın şarkılarının türkiye  ilk seslendirilişini gerçekleştirdi son olarak şef ibrahimyazıcı yönetimindeki bilkent senfoni orkestrası eşliğinde say  ın  nazım  adlı eserinin cd dvd kayıtlarını fazılsay gencoerkal zuhalolcay birlikte gerçekleştirdi  bana mastika  yı ç

In [211]:
# Burada indexleri tersine çevirerek kontrol ediyoruz
reverse_word_index= dict([(idx , word) for  (word,idx) in word_index.items()])

In [212]:
def decode(sequence):
  return " ".join([reverse_word_index.get(idx , "?") for idx in sequence])

In [213]:
decoded_text = decode(train_sequences[10])
print(train_sequences[10])
print(decoded_text)

[9451, 26694, 50463, 21712, 239, 165, 68, 434, 212, 207, 2, 63, 6319, 2, 3128, 4565, 15842, 26695, 3564, 40, 272, 345, 15843, 1155, 2883, 2, 1037, 50464, 2, 53, 7, 50465, 50466, 6, 211, 21713, 26696, 50467, 266, 240, 50468, 50469, 50470, 1, 4566, 554, 1038, 6319, 2, 12440, 3681, 4383, 50471, 50472, 2, 10, 7, 50473, 8105, 6, 50474, 5192, 2286, 264, 19, 50475, 50476, 54, 34495, 446, 4056, 8, 7, 50477, 1255, 50478, 50479, 6, 1538, 7127, 20, 5689, 6709, 4051, 2, 9, 7, 5690, 6, 7573, 4051, 4959, 34495, 77, 12441, 7, 18405, 50480, 1255, 8745, 15844, 6, 21714, 4051, 77, 1, 34496, 93, 9443, 2, 9, 26697, 3, 2, 19, 50481, 2203, 26, 5, 5423, 21715, 4553, 18406, 5438, 4057, 2717, 9443, 2, 9, 7, 5690, 6, 211, 10242, 4567, 11240, 18407, 4051, 4959, 50482, 77, 2203, 7, 533, 50483, 2, 443, 50484, 6, 7, 52, 2, 1, 906, 6, 12442, 2, 9, 7, 50485, 6, 531, 1561, 26694, 2, 227, 40, 272, 4058, 50, 354, 50486, 34497, 387, 1451, 34497, 387]
güvenç dağüstün urban lounge 18 11 2012 22 00 ankara  dan viyana  uza

In [214]:
#LSTM modeli yapıyoruz.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Model oluşturma
model = Sequential()
model.add(Embedding(input_dim=num_unique_words, output_dim=100, input_length=20))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=7, activation='softmax'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 100)           11089800  
                                                                 
 lstm_12 (LSTM)              (None, 20, 128)           117248    
                                                                 
 lstm_13 (LSTM)              (None, 128)               131584    
                                                                 
 dense_16 (Dense)            (None, 7)                 903       
                                                                 
Total params: 11,339,535
Trainable params: 11,339,535
Non-trainable params: 0
_________________________________________________________________


In [215]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [216]:
from sklearn.preprocessing import OneHotEncoder

# Tüm kategorileri birleştirerek unique kategorileri belirleyin
all_labels = np.concatenate((train_labels, val_labels), axis=0)
unique_labels = np.unique(all_labels)

# One-hot encoder oluşturun ve unique kategorileri ileti
encoder = OneHotEncoder(categories=[unique_labels])
train_labels_encoded = encoder.fit_transform(train_labels.reshape(-1, 1)).toarray()
val_labels_encoded = encoder.transform(val_labels.reshape(-1, 1)).toarray()


In [217]:
len(train_padded) , len(val_padded)

(3185, 1715)

In [ ]:
history = model.fit(train_padded, train_labels_encoded, validation_data=(val_padded, val_labels_encoded), epochs=10)


Epoch 1/10
100/100 [==============================] - 62s 424ms/step - loss: 1.5719 - accuracy: 0.3780 - val_loss: 0.9779 - val_accuracy: 0.6799
Epoch 2/10
100/100 [==============================] - 40s 397ms/step - loss: 0.4657 - accuracy: 0.8465 - val_loss: 0.7025 - val_accuracy: 0.7796
Epoch 3/10
100/100 [==============================] - 40s 398ms/step - loss: 0.1259 - accuracy: 0.9642 - val_loss: 0.7012 - val_accuracy: 0.7971
Epoch 4/10
100/100 [==============================] - 41s 410ms/step - loss: 0.0501 - accuracy: 0.9881 - val_loss: 0.8060 - val_accuracy: 0.7773
Epoch 5/10
100/100 [==============================] - 39s 392ms/step - loss: 0.0387 - accuracy: 0.9909 - val_loss: 0.8754 - val_accuracy: 0.7819
Epoch 6/10
100/100 [==============================] - 40s 398ms/step - loss: 0.0294 - accuracy: 0.9937 - val_loss: 0.8913 - val_accuracy: 0.7866
Epoch 7/10
100/100 [==============================] - 39s 394ms/step - loss: 0.0258 - accuracy: 0.9943 - val_loss: 1.0621 - val_ac